# Factory Demo

In [2]:
from patterns.Factory_InstrumentTypes import InstrumentFactory

instruments = InstrumentFactory.load_from_csv('inputs/instruments.csv')

print(instruments[0].instrument_type)
print(instruments[0].symbol)

custom_data = {
    'symbol': 'TSLA',
    'type': 'stock',
    'price': '250.00',
    'sector': 'Automotive',
    'issuer': 'Tesla Inc.'
}

custom_stock = InstrumentFactory.create_instrument(custom_data)

print("--")

print(custom_stock.symbol)


stock
AAPL
--
TSLA


# Singleton

In [3]:
from patterns.Singleton_ConfigAccess import Config

config = Config()

# Access
# print(config.config)

print(config.config.get('log_level'))
# print(config.config.get('data_path'))
# print(config.config.get('report_path'))

# Check:
s1 = Config()
s2 = Config()

print(s1 is s2)  # Output: True (both variables point to the same instance)

INFO
True


# Builder

In [4]:
from patterns.Builder_PortfolioBuilder import PortfolioBuilder, Director

director = Director()

# from portfolio_structure.json
portfolio = director.build_portfolio()


print(portfolio.name)
print(portfolio.owner)
print(portfolio.positions)
# subport access
print("--")
print(portfolio.subportfolios)
print(portfolio.subportfolios[0]['subportfolio'].positions)
# print(portfolio.subportfolios[0].positions)



Main Portfolio
sdonadio
[{'symbol': 'AAPL', 'quantity': 100, 'price': 172.35}, {'symbol': 'MSFT', 'quantity': 50, 'price': 328.1}]
--
[{'name': 'Index Holdings', 'subportfolio': <patterns.Builder_PortfolioBuilder.Portfolio object at 0x10d6c1f90>}]
[{'symbol': 'SPY', 'quantity': 20, 'price': 430.5}]


# Decorator

In [5]:
from patterns.Factory_InstrumentTypes import Stock
from Decorator_Analytics import VolatilityDecorator, BetaDecorator, DrawdownDecorator

# Create a simple Stock instrument (we don't modify the base class)
stock = Stock({'symbol': 'AAPL', 'price': 169.89, 'type': 'stock'})

# Stack decorators: volatility -> beta -> drawdown
decorated = DrawdownDecorator(BetaDecorator(VolatilityDecorator(stock)))

metrics = decorated.get_metrics()
print(f"Metrics for {stock.symbol}:")
for k, v in metrics.items():
    print(f"  {k}: {v}")


Metrics for AAPL:
  volatility: 0.0024507231865002555
  beta: -0.011246134064548429
  max_drawdown: 0.7361397311353155


# Adapter

In [6]:
import importlib
# from Adapter_DataLoader import YahooFinanceAdapter, BloombergXMLAdapter
import Adapter_DataLoader

importlib.reload(Adapter_DataLoader)

yahoo_adapter = Adapter_DataLoader.YahooFinanceAdapter()

print(yahoo_adapter.get_data('AAPL').symbol)
print(yahoo_adapter.get_data('AAPL').price)
print(yahoo_adapter.get_data('AAPL').timestamp)

bloomberg_adapter = Adapter_DataLoader.BloombergXMLAdapter()

print(bloomberg_adapter.get_data('MSFT').symbol)
print(bloomberg_adapter.get_data('MSFT').price)
print(bloomberg_adapter.get_data('MSFT').timestamp)




AAPL
172.35
2025-10-01 09:30:00+00:00
MSFT
328.1
2025-10-01 09:30:00+00:00


# Composite

In [7]:
from patterns.Composite_PortModel import build_portfolio_from_json

portfolio = build_portfolio_from_json('inputs/portfolio_structure.json')

print(portfolio.name)
print(portfolio.get_value())

for pos in portfolio.get_positions():
    print(pos)


Main Portfolio
42250.0
{'symbol': 'AAPL', 'quantity': 100, 'price': 172.35}
{'symbol': 'MSFT', 'quantity': 50, 'price': 328.1}
{'symbol': 'SPY', 'quantity': 20, 'price': 430.5}


# Strategy

In [8]:
import importlib
import engine

importlib.reload(engine)

from patterns.Strategy_SignalGen import MeanReversionStrategy
from patterns.Strategy_SignalGen import BreakoutStrategy

# Initialize engine
engine = engine.BacktestEngine(initial_capital=100000)

# Load market data
print("Loading market data...")
df = engine.load_market_data()

# Get unique symbols
symbols = df['symbol'].unique()
print(f"Found symbols: {symbols}")

# Initialize strategies
mean_reversion = MeanReversionStrategy()
mean_reversion.load_params()

# ------------------------------------------------------------
# strategy.set_strategy(mean_reversion)
# ------------------------------------------------------------

breakout = BreakoutStrategy()
breakout.load_params()

# Run backtest for each symbol (one at a time per Strategy pattern requirement)
for symbol in symbols:
    # Test Mean Reversion Strategy
    engine.backtest_strategy(mean_reversion, symbol, df)
    
    # Optionally test Breakout Strategy
    # engine.backtest_strategy(breakout, symbol, df)

engine.backtest_strategy(mean_reversion, 'AAPL', df)


# Overall summary
print(f"\n{'='*80}")
print("BACKTEST SUMMARY")
print(f"{'='*80}")
print(f"Total trades executed: {len(engine.trades)}")
print(f"Final cash: ${engine.cash:,.2f}")
print(f"Open positions: {sum(1 for p in engine.positions.values() if p.quantity > 0)}")

Loading market data...
Found symbols: ['AAPL' 'MSFT' 'SPY']

Backtesting MeanReversionStrategy on AAPL
INSUFFICIENT SHARES: Cannot sell AAPL | Available: 0
INSUFFICIENT SHARES: Cannot sell AAPL | Available: 0
BUY:  1 share(s) of AAPL at $112.64 | Cash: $99887.36 | position: 1
SELL: 1 share(s) of AAPL at $111.88 | PnL: $111.88 | Cash: $99999.24 | position: 0
INSUFFICIENT SHARES: Cannot sell AAPL | Available: 0
INSUFFICIENT SHARES: Cannot sell AAPL | Available: 0
INSUFFICIENT SHARES: Cannot sell AAPL | Available: 0
INSUFFICIENT SHARES: Cannot sell AAPL | Available: 0
BUY:  1 share(s) of AAPL at $104.09 | Cash: $99895.15 | position: 1
SELL: 1 share(s) of AAPL at $109.92 | PnL: $109.92 | Cash: $100005.07 | position: 0
BUY:  1 share(s) of AAPL at $101.24 | Cash: $99903.83 | position: 1
BUY:  1 share(s) of AAPL at $101.20 | Cash: $99802.63 | position: 2
BUY:  1 share(s) of AAPL at $100.62 | Cash: $99702.01 | position: 3
BUY:  1 share(s) of AAPL at $99.38 | Cash: $99602.63 | position: 4
SELL:

# Observer

In [9]:
import importlib
import engine
from patterns.Observer_SignalNotification import LoggerObserver, AlertObserver
from patterns.Strategy_SignalGen import MeanReversionStrategy

importlib.reload(engine)

# Initialize engine
engine = engine.BacktestEngine(initial_capital=100000)

# Load market data
print("Loading market data...")
df = engine.load_market_data()

# Initialize observers
logger = LoggerObserver(log_level='INFO')
alert = AlertObserver()

# Attach observers to both engine and strategy
engine.publisher.attach(logger)
engine.publisher.attach(alert)

# Create strategy
strategy = MeanReversionStrategy()
strategy.load_params()

# Attach observers to strategy as well
strategy.publisher.attach(logger)
strategy.publisher.attach(alert)

print(f"\n{'='*80}")
print("OBSERVER PATTERN DEMONSTRATION")
print(f"{'='*80}\n")
print(f"Observers attached: {len(engine.publisher.observers)}")
print(f"Dynamic registration demonstrated\n")

# Run backtest
engine.backtest_strategy(strategy, 'AAPL', df)

# Print observer statistics
print(f"\n{'='*80}")
print("OBSERVER STATISTICS")
print(f"{'='*80}")
print(f"LoggerObserver: {logger.get_log_count()} signals logged")
print(f"AlertObserver: {alert.get_alert_count()} alerts triggered")
print(f"{'='*80}\n")

# Demonstrate dynamic detach
engine.publisher.detach(logger)
print(f"LoggerObserver detached. Remaining observers: {len(engine.publisher.observers)}")

# Run another symbol without logger
engine.backtest_strategy(strategy, 'MSFT', df)

Loading market data...
Observer LoggerObserver attached
Observer AlertObserver attached
Observer LoggerObserver attached
Observer AlertObserver attached

OBSERVER PATTERN DEMONSTRATION

Observers attached: 2
Dynamic registration demonstrated


Backtesting MeanReversionStrategy on AAPL
[INFO] Signal #1: SELL signal for AAPL at $134.69 using MeanReversionStrategy - Action: SELL
[INFO] Signal #2: INSUFFICIENT_POSITION signal for AAPL at $134.69 using Unknown - Action: SELL
ALERT #1: INSUFFICIENT POSITION
   Cannot sell 1 share(s) of AAPL at $134.69
   Available: 0 share(s)
INSUFFICIENT SHARES: Cannot sell AAPL | Available: 0
[INFO] Signal #3: SELL signal for AAPL at $135.11 using MeanReversionStrategy - Action: SELL
[INFO] Signal #4: INSUFFICIENT_POSITION signal for AAPL at $135.11 using Unknown - Action: SELL
ALERT #2: INSUFFICIENT POSITION
   Cannot sell 1 share(s) of AAPL at $135.11
   Available: 0 share(s)
INSUFFICIENT SHARES: Cannot sell AAPL | Available: 0
[INFO] Signal #5: BUY sign

# Command

In [10]:
from engine import BacktestEngine, Position
from datetime import datetime
from patterns.Command_TradeExecution import ExecuteOrderCommand

# Create engine
engine = BacktestEngine(initial_capital=100000)

# Execute a BUY order
command1 = ExecuteOrderCommand(
    engine, 
    timestamp=datetime.now(), 
    symbol='AAPL', 
    action='BUY', 
    price=150.00, 
    quantity=1
)
engine.command_invoker.execute_command(command1)

# Get position safely
aapl_pos = engine.positions.get('AAPL')
quantity = aapl_pos.quantity if aapl_pos else 0
print(f"After BUY - Cash: ${engine.cash:.2f}, AAPL position: {quantity}")

# Undo the order
engine.command_invoker.undo()
aapl_pos = engine.positions.get('AAPL')
quantity = aapl_pos.quantity if aapl_pos else 0
print(f"After UNDO - Cash: ${engine.cash:.2f}, AAPL position: {quantity}")

# Redo the order
engine.command_invoker.redo()
aapl_pos = engine.positions.get('AAPL')
quantity = aapl_pos.quantity if aapl_pos else 0
print(f"After REDO - Cash: ${engine.cash:.2f}, AAPL position: {quantity}")

# Execute a SELL order (assuming you still have the position)
command2 = ExecuteOrderCommand(
    engine, 
    timestamp=datetime.now(), 
    symbol='AAPL', 
    action='SELL', 
    price=155.00, 
    quantity=1
)
engine.command_invoker.execute_command(command2)

aapl_pos = engine.positions.get('AAPL')
quantity = aapl_pos.quantity if aapl_pos else 0
print(f"After SELL - Cash: ${engine.cash:.2f}, AAPL position: {quantity}")

# Undo the SELL
engine.command_invoker.undo()
aapl_pos = engine.positions.get('AAPL')
quantity = aapl_pos.quantity if aapl_pos else 0
print(f"After UNDO SELL - Cash: ${engine.cash:.2f}, AAPL position: {quantity}")

After BUY - Cash: $99850.00, AAPL position: 1
After UNDO - Cash: $100000.00, AAPL position: 0
After REDO - Cash: $99850.00, AAPL position: 1
After SELL - Cash: $100005.00, AAPL position: 0
After UNDO SELL - Cash: $99850.00, AAPL position: 1
